In [1]:
import json
import re
import spacy
from spacy.tokens import DocBin

# Explore data

In [2]:
with open('data/dataset_lcquad2.json') as df:
    lcquad2_data = json.load(df)
lcquad2_data.keys()

dict_keys(['questions', 'train', 'valid'])

In [4]:
print(lcquad2_data['questions'][2]['natural_language_question'])
# print(lcquad2_data['questions'][0]['query_answer'][0]['sparql_query'])
print(lcquad2_data['questions'][2]['query_answer'][0]['entities'])
# print(lcquad2_data['questions'][0]['query_answer'][0]['slots'])
# print(lcquad2_data['questions'][0]['query_answer'][0]['sparql_template'])

Is Harrelson the real family name of Woody Harrelson?
[{'label': 'Woody_Harrelson', 'entity': 'wd:Q201279'}, {'label': 'Harrelson', 'entity': 'wd:Q21450338'}]


In [ ]:
print(len(lcquad2_data['questions']))

In [ ]:
slot_types = set()

for q in lcquad2_data['questions']:
    for s in q['query_answer'][0]['slots']:
        slot_types.add(s['slot'])
slot_types

In [ ]:
for q in lcquad2_data['questions']:
    for s in q['query_answer'][0]['slots']:
        if s['slot'] == '<num>':
            print(q['natural_language_question'])
            print(q['query_answer'])

# Get data into spacy format

In [ ]:
# Construct dataset for spacy custom NER training
all_questions = []

for q in lcquad2_data['questions']:

    qi = q['natural_language_question']

    ents = []
    for e in q['query_answer'][0]['entities']:
        s = qi.find(re.sub('_', ' ', e['label']))

        if s > -1:
            e = s + len(e['label'])            
            ents.append((s, e, entity_label))

    if len(ents) > 0:
        all_questions.append((qi, ents))

print(len(all_questions))
all_questions[:2]

In [ ]:
all_questions[44]

In [ ]:
nlp = spacy.blank("en")
# training_data = [
#   ("Tokyo Tower is 333m tall.", [(0, 11, "BUILDING")]),
# ]

# the DocBin will store the example documents
db = DocBin()
invalid = []
for i, (text, annotations) in enumerate(all_questions):
    if i % 1000 == 0:
        print(i)
        
    try:
        doc = nlp(text)
        ents = []
        for start, end, label in annotations:
            span = doc.char_span(start, end, label=label)
            ents.append(span)
        doc.ents = ents
        db.add(doc)
    except:
        invalid.append(i)
print(f'{len(invalid)} were invalid')
db.to_disk("data/train.spacy") # train dev

# Train Spacy NER model

In [ ]:
!python -m spacy init config --lang en --pipeline ner data/config.cfg --force

In [ ]:
!python -m spacy train data/config.cfg --output data/ --paths.train data/train.spacy --paths.dev data/dev.spacy --training.eval_frequency 10 --training.max_steps 500

In [ ]:
nlp1 = spacy.load(R"data/model-last") #load the best model
doc = nlp1("Does FC Barcelona have Juan Jose Ibarretxe as a chairperson") # input sample tex

spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter

# nlp1 = spacy.load(R"/content/ner_demo/training/model-last") #load the best model
# doc = nlp1("I like CA") # input sample tex

# spacy.displacy.render(doc, style="ent", jupyter=True)

In [ ]:
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)